<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following example shows that an STFT can be indeed unitary:


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import ShortTimeFFT, windows

m, hop, std = 36, 8, 5
desired_win = windows.gaussian(m, std, sym=True)
SFT = ShortTimeFFT.from_win_equals_dual(desired_win, hop, fs=1/m,
                                        fft_mode='twosided',
                                        scale_to='unitary')
np.allclose(SFT.dual_win, SFT.win * SFT.m_num)  # check if STFT is unitary

True

In [ ]:
x1, x2 = np.tile([-1, -1, 1, 1], 5), np.tile([1, -1, -1, 1], 5)
np.sum(x1*x2) # scalar product is zero -> orthogonal signals

0

In [ ]:
np.sum(x1**2)  # scalar product of x1 with itself

20

In [ ]:
Sx11, Sx12 = SFT.spectrogram(x1), SFT.spectrogram(x1, x2)
np.sum(Sx12)  # STFT scalar product is also zero

-4.163336342344337e-16+0j  # may vary

In [ ]:
np.sum(Sx11)  # == np.sum(x1**2)

# Do the plotting:

19.999999999999996  # may vary

In [ ]:
fg1, (ax11, ax12) = plt.subplots(1, 2, tight_layout=True, figsize=(8, 4))
s_fac = np.sqrt(SFT.mfft)
_ = fg1.suptitle(f"Scaled Unitary Window of {m} Sample Gaussian with " +
                 rf"{hop=}, $\sigma={std}$, Scale factor: {s_fac}")
ax11.set(ylabel="Amplitude", xlabel="Samples", xlim=(0, m))
ax12.set(xlabel="Frequency Bins", ylabel="Magnitude Spectrum",
         xlim=(0, 15), ylim=(1e-5, 1.5))
u_win_str = rf"Unitary $\times{s_fac:g}$"
for x_, n_ in zip((desired_win, SFT.win*s_fac), ('Desired', u_win_str)):
    ax11.plot(x_, '.-', alpha=0.5, label=n_)
    X_ = np.fft.rfft(x_) / np.sum(abs(x_))
    ax12.semilogy(abs(X_), '.-', alpha=0.5, label=n_)
for ax_ in (ax11, ax12):
    ax_.grid(True)
    ax_.legend()
plt.show()

Note that ``fftmode='twosided'`` is used, since we need sum over the complete
time frequency plane. Due to passing ``scale_to='unitary'`` the window
``SFT.win`` is scaled by ``1/np.sqrt(SFT.mfft)``. Hence, ``SFT.win`` needs to
be scaled by `s_fac` in the plot above.
